In [4]:
from cobra import Metabolite
from cobra import Reaction
import cameo
import cobra

In [5]:
#importing iJO1366 model
model = cameo.models.bigg.iJO1366
model.reactions.EX_co2_e.lower_bound = 0

#metabolite declaration
aacoa = model.metabolites.get_by_id('aacoa_c')
nadph = model.metabolites.get_by_id('nadph_c')
nadp = model.metabolites.get_by_id('nadp_c')
hbcoa = Metabolite('3hbcoa__R_c', formula = 'C25H38N7O18P3S', name = '3-hydroxybutyryl-CoA', charge = -4, compartment = 'c')
coa = model.metabolites.get_by_id('coa_c')
hbal3 = Metabolite('hbal3_c', formula = 'C4H8O2', name = '3-hydroxybutanal', compartment = 'c')
nadh = model.metabolites.get_by_id('nadh_c')
nad = model.metabolites.get_by_id('nad_c')
H = model.metabolites.get_by_id('h_c')
BD_13 = Metabolite('13btdl_c', formula = 'C4H10O2', name = '1,3-butanediol', compartment = 'c')


In [6]:
#reaction 1: aacoa + nadph + H <-> hbcoa + nadp
reaction1 = Reaction('aacoa_hbcoa')
reaction1.name = 'acetoacetyl-CoA to 3-hydroxybutyryl-CoA'
reaction1.lower_bound = 0
reaction1.upper_bound = 1000
reaction1.add_metabolites({aacoa: -1.0, nadph: -1.0, H: -1.0, hbcoa: 1.0, nadp: 1.0})
model.add_reaction(reaction1)

#reaction 2: hbcoa + nadph + H <=> hbal3 + nadp + coa
reaction2 = Reaction('hbcoa_hbal3')
reaction2.name = '3-hydroxybutyryl-CoA to 3-hydroxybutanal'
reaction2.lower_bound = 0
reaction2.upper_bound = 1000
reaction2.add_metabolites({hbcoa: -1.0, nadph: -1.0, H: -1.0, hbal3: 1.0, nadp: 1.0, coa: 1.0})
model.add_reaction(reaction2)

#reaction 3: hbal3 + nadh + H -> BD_13 + nad
reaction3 = Reaction('hbal3_bd_13')
reaction3.name = '3-hydroxybutanal to 1,3-butanediol'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({hbal3: -1.0, nadh: -1.0, H: -1.0, BD_13: 1.0, nad: 1.0})
model.add_reaction(reaction3)

#reaction 4: BD_13 <=>
reaction4 = Reaction('EX_13btdl_e')
reaction4.name = '1,3-butanediol consumption'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({BD_13: -1.0})
model.add_reaction(reaction4)

In [7]:
#optimum flux calculation
model.objective = 'EX_13btdl_e'
max_flux = 0
max_flux=(model.optimize(objective_sense='maximize').objective_value)
print(max_flux)

10.804958677685951


In [8]:
model.summary()

In [13]:
cobra.io.write_sbml_model(model, "1_3_butanediol.xml")